In [ ]:
import heapq as hq
from random import shuffle

"""
class Passenger:
    def __init__(self, BM, exit_time):
        self.bm = BM
        self.exit = exit_time
    def __lt__(self, other):
        return (self.exit, self.bm) < \
               (other.exit, other.bm)

class BM:
    def __init__(self, passenger, capacity, stop, itinerary):
        self.passenger = passenger
        self.capacity = capacity
        self.stop = stop
        self.itinerary = itinerary
    def __le__(self, other):
        pass
"""

class Event:
    def __init__(self, time, action, bm):
        self.time = time
        self.action = action
        self.bm = bm
    def __lt__(self, other):
        return (self.time, self.bm) < (other.time, other.bm)
    def __le__(self, other):
        return (self.time, self.bm) <= (other.time, other.bm)

In [ ]:
def solve(eventq, passengers):
    while eventq:
        if [len(passengers[i]) == 0 for i in range(len(passengers))] == [True]*len(passengers):
            return 'Ende'
        event = hq.heappop(eventq)
        bm = bms[event.bm]
        current_node = bm['itinerary'][bm['stop'] % len(bm['itinerary'])]
        next_node = bm['itinerary'][(bm['stop']+1) % len(bm['itinerary'])]
        print('Fahrzeug {} ist am Knoten {}, der nähste halt {}'.format(bm['name'],current_node, next_node))
        if event.action == 'load':
            free = bm['capacity'] - bm['n_passengers']
            print('Fahrzeug {} load'.format(bm['name']))
            print('Fahrzeug {} hat {} Plätze frei'.format(bm['name'],free))
            while bm['n_passengers'] < bm['capacity']:
                waiting = [idx for idx, passenger in enumerate(passengers[current_node]) if passenger['station'] in bm['itinerary']]
                print('Warten: {}'.format(waiting))
                if not waiting:
                    break
                passenger = passengers[current_node].pop(waiting[0])
                print('Passagier:', passenger)
                bm['n_passengers'] += 1
                hq.heappush(eventq, Event(event.time + passenger['exit_time'],'exit', event.bm))
                print('time:', event.time+passenger['exit_time'])
            hq.heappush(eventq, Event(event.time+125, 'load', event.bm))
            bm['stop'] += 1
            # Generate events for exiting passengers
        elif event.action == 'exit':
            print('Fahrzeug {}, Passagiere exit'.format(bm['name']))
            print('Es sind {} Passagiere in der Bahn'.format(bm['n_passengers']))
            bm['n_passengers'] -= 1
        else:
            raise Exception('Unknown event action')



In [ ]:
def shuffle0(passengers):
    for i in range(len(passengers)):
        x = [j for j in range(len(passengers[i]))]
        shuffle(x)
        passengers[i] = [passengers[i][k] for k in x]

In [ ]:
passengers = [#
    [{'station': 1, 'exit_time': 60}]*1 + [{'station': 2, 'exit_time':120}]*5,#
    [{'station': 0, 'exit_time':110}]*5 + [{'station': 2, 'exit_time': 60}]*12,#
    [{'station': 0, 'exit_time': 50}]*0 + [{'station': 1, 'exit_time':110}]*0#
              ]#
bms = [{'n_passengers': 0, 'capacity': 10, 'stop': 0, 'itinerary': [0, 1], 'name': 0},
       {'n_passengers': 0, 'capacity': 10, 'stop': 0, 'itinerary': [0, 1, 2, 1], 'name': 1, 't_times': [60, 50, 55]}]

# shuffle passengers
shuffle0(passengers)

event0 = Event(0,'load',0)
event1 = Event(3,'load',1)
eventq = []
hq.heappush(eventq, event0)
hq.heappush(eventq, event1)

In [ ]:
solve(eventq, passengers)